In [1]:
import functools

import numpy as np

import brainscore_vision
from brainio.assemblies import BehavioralAssembly, DataAssembly, walk_coords
from brainio.stimuli import StimulusSet
from brainscore_vision.model_helpers.utils import make_list
from brainscore_vision.benchmark_helpers.screen import place_on_screen
from brainscore_vision.model_helpers.activations import PytorchWrapper
from brainscore_vision.model_helpers.brain_transformation import ModelCommitment
from brainscore_vision.model_interface import BrainModel

/Users/linussommer/miniforge3/envs/bs_old/lib/python3.7/site-packages/brainscore_core/metrics/__init__.py:16: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


In [2]:
def pytorch_custom():
    import torch
    from torch import nn
    from brainscore_vision.model_helpers.activations.pytorch import load_preprocess_images

    class MyModel(nn.Module):
        def __init__(self):
            super(MyModel, self).__init__()
            np.random.seed(0)
            torch.random.manual_seed(0)
            self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3)
            self.relu1 = torch.nn.ReLU()
            linear_input_size = np.power((224 - 3 + 2 * 0) / 1 + 1, 2) * 2
            self.linear = torch.nn.Linear(int(linear_input_size), 1000)
            self.relu2 = torch.nn.ReLU()

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = x.view(x.size(0), -1)
            x = self.linear(x)
            x = self.relu2(x)
            return x

    preprocessing = functools.partial(load_preprocess_images, image_size=224)
    return PytorchWrapper(model=MyModel(), preprocessing=preprocessing)

In [3]:
# create model
activations_model = pytorch_custom()
layers = ["relu2"]

# create brain model
brain_model = ModelCommitment(
    identifier=activations_model.identifier, 
    activations_model=activations_model, 
    layers=[None], 
    behavioral_readout_layer='relu2')

assy = brainscore_vision.load_dataset(f'Hebart2023')
triplets = place_on_screen(
    stimulus_set=assy.stimulus_set,
    target_visual_degrees=brain_model.visual_degrees(),
    source_visual_degrees=8)
    
features = activations_model(triplets, layers=layers)
features = features.transpose('presentation', 'neuroid')

In [4]:
def calculate_similarity_matrix(features, similarity_measure='dot'):
   features = features.transpose('presentation', 'neuroid')
   values = features.values
   if similarity_measure == 'dot':
      similarity_matrix = np.dot(values, np.transpose(values))
   elif similarity_measure == 'cosine':
      row_norms = np.linalg.norm(values, axis=1).reshape(-1, 1)
      norm_product = np.dot(row_norms, row_norms.T)
      dot_product = np.dot(values, np.transpose(values))
      similarity_matrix = dot_product / norm_product
   else:
      raise ValueError(
      f"Unknown similarity_measure {similarity_measure} -- expected one of 'dot' or 'cosine'")

   similarity_matrix = DataAssembly(similarity_matrix, coords={
        **{f"{coord}_left": ('presentation_left', values) for coord, _, values in
           walk_coords(features['presentation'])},
        **{f"{coord}_right": ('presentation_right', values) for coord, _, values in
           walk_coords(features['presentation'])}
   }, dims=['presentation_left', 'presentation_right'])
   return similarity_matrix

In [5]:
def calculate_choices(similarity_matrix, triplets):
    choice_predictions = []
    for triplet in triplets:
        i, j, k = triplet
        sims = similarity_matrix[i, j], similarity_matrix[i, k],  similarity_matrix[j, k]
        idx = triplet[2 - np.argmax(sims)]
        choice_predictions.append(idx)
    # TODO return as DataAssembly
    return choice_predictions

In [6]:
triplets = brainscore_vision.load_dataset(f'Hebart2023')
triplets = np.array([triplets['image_1'], triplets['image_2'], triplets['image_3']]).T
sample = triplets[:10]
sim = calculate_similarity_matrix(features, similarity_measure='cosine')
choices = calculate_choices(similarity_matrix=sim, triplets=sample)

In [9]:
print(sample)
print(choices)

[[ 112 1459  632]
 [1278 1561  792]
 [ 796 1660  413]
 [  27  671  320]
 [1448 1531  324]
 [1594  280 1825]
 [1381 1522   14]
 [1528  526  208]
 [1491 1336 1850]
 [1494  171 1113]]
[1459, 1278, 1660, 320, 1531, 1825, 14, 1528, 1850, 1113]
